# Code

In [1]:
its_jupyter_notebook = True

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle
import math
import torch
import random
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA, IncrementalPCA
import seaborn as sns
from torch.utils.data import DataLoader
# if its_jupyter_notebook:
#     sys.argv = [''] #Remove this if it's not a jupyter notebook!

import sys
sys.path.insert(0, '..')
from config import *

In [3]:
N_RNA = 1_000
N_RNA_BATCH_SIZE = 5

In [4]:
df_nt = pd.read_csv(os.path.join(metadata_dir, f'df_nt.csv'))
df_genes_nt = pd.read_csv(os.path.join(metadata_dir, f'df_genes_nt.csv'))

/tmp/ipykernel_83903/2145422499.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nt = pd.read_csv(os.path.join(metadata_dir, f'df_nt.csv'))


In [5]:
subset_train_nt = os.path.join(rna_rna_files_dir, f"gene_pairs_training_nt.txt")
subset_test_nt = os.path.join(rna_rna_files_dir, f"gene_pairs_test_nt.txt")
subset_val_nt = os.path.join(rna_rna_files_dir, f"gene_pairs_val_nt.txt")

with open(subset_train_nt, "rb") as fp:  # Unpickling
    subset_train_nt = pickle.load(fp)

In [6]:
# OLD SET
df_nt = pd.read_csv(os.path.join(metadata_dir, 'selected_with_bias', f'df_nt.csv'))
df_genes_nt = pd.read_csv(os.path.join(metadata_dir, 'selected_with_bias',f'df_genes_nt.csv'))

subset_train_nt = os.path.join(rna_rna_files_dir, 'selected_with_bias', f"gene_pairs_training_nt.txt")

with open(subset_train_nt, "rb") as fp:  # Unpickling
    subset_train_nt = pickle.load(fp)

/tmp/ipykernel_83903/3973500347.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nt = pd.read_csv(os.path.join(metadata_dir, 'selected_with_bias', f'df_nt.csv'))


In [7]:
train_subset = df_nt[df_nt.couples.isin(subset_train_nt)]
list_to_sample_from = list(set(train_subset.gene1).union(train_subset.gene2))
random.shuffle(list_to_sample_from)
selected_genes = list_to_sample_from[:N_RNA]

In [8]:
ipca = IncrementalPCA(n_components=N_PCA)

In [9]:
start_time = time.time()

features = []
n_loaded = 0

for i in tqdm(range(N_RNA)):
    arr = np.load(os.path.join(embedding_dir, '32', f'{selected_genes[i]}.npy')) #array is [len_rna, 2560]
    features.append(arr)
    n_loaded +=1
    if n_loaded == N_RNA_BATCH_SIZE:
        features_concatenated = np.concatenate(features, axis=0)  # array is [N_RNA_BATCH_SIZE*len_rna, 2560]
        ipca.partial_fit(features_concatenated)
        features = []
        n_loaded = 0
        
# Fit the remaining data
if n_loaded > 0:
    features_concatenated = np.concatenate(features, axis=0)
    ipca.partial_fit(features_concatenated)
    
print(f"Total time to fit PCA: {(time.time()-start_time)/60} minutes")

  0%|          | 0/1000 [00:00<?, ?it/s]

Total time to fit PCA: 8.947395654519399 minutes


In [10]:
explained_variance_ratio = ipca.explained_variance_ratio_
print("Explained variance ratio:", explained_variance_ratio)

cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)
print("Cumulative explained variance ratio:", cumulative_explained_variance_ratio)

Explained variance ratio: [0.08193161 0.03407654 0.02770195 0.02459305 0.02116112 0.01578076
 0.00899    0.00805853 0.00690973 0.00649344 0.00606851 0.00594266
 0.00544771 0.00531856 0.00505746 0.00482862 0.00473687 0.00465819
 0.00456554 0.00453264 0.00442819 0.00427909 0.00424589 0.00421024
 0.00413137 0.00411668 0.00405806 0.00402244 0.0039098  0.00384794
 0.00376381 0.00367666 0.00354024 0.00340209 0.00332847 0.00318732
 0.00310725 0.00308743 0.00305808 0.00297705 0.00292751 0.00284555
 0.00280776 0.00266212 0.00258059 0.00251706 0.00249704 0.0024711
 0.00244028 0.00240521 0.00233639 0.00229464 0.00228609 0.00223124
 0.00220105 0.00220072 0.00217607 0.00216676 0.00211107 0.00207888
 0.00205043 0.00202099 0.00197439 0.00195825 0.00192622 0.00188758
 0.00186703 0.00184314 0.00181997 0.00180343 0.00176442 0.00174739
 0.0017214  0.00168492 0.00167175 0.00165431 0.00164493 0.00163105
 0.00161849 0.00160777 0.00159993 0.00158492 0.0015679  0.0015591
 0.00154229 0.0015262  0.00152235 0.00

In [11]:
start_time = time.time()

for i in tqdm(range(N_RNA)):
    arr = np.load(os.path.join(embedding_dir, '32', f'{selected_genes[i]}.npy')) #array is [len_rna, 2560]
    arr = ipca.transform(features_concatenated) #save in the folder
    
print(f"Total time to transform the data PCA: {(time.time()-start_time)/60} minutes")

  0%|          | 0/1000 [00:00<?, ?it/s]

Total time to transform the data PCA: 1.5221766471862792 minutes
